This model uses CNNs to classify audio input into instrument classes.

General Procedure:
	- Use STFTs to transform audio input into spectral graphs
	- Use spectral graphs as "images" to classify into instrument classes

Data Format:
 	- Data is stored in tfrecord form obtained from the Nsynth dataset: https://magenta.tensorflow.org/datasets/nsynth

Based off of aymericdamien's TensorFlow examples: https://github.com/aymericdamien/TensorFlow-Examples

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib.framework.python.ops import audio_ops
from tensorflow.python import debug as tf_debug
import matplotlib.pyplot as plt
import numpy as np

# For debugging on Windows 10
from pyreadline import Readline
readline = Readline()

In [2]:
# Training Parameters
LEARNING_RATE = 0.001
NUM_STEPS = 10000
BATCH_SIZE = 32

# Network Parameters
NUM_INPUT = 16384   # spectrogram data input (img shape: 128*128)
NUM_CLASSES = 11    # total instrument classes
DROPOUT = 0.25      # Dropout, probability to drop a unit

# Data paths
TRAINING_DATA = ["E:/NSynth/nsynth-train-0.tfrecord", 
                 "E:/NSynth/nsynth-train-1.tfrecord", 
                 "E:/NSynth/nsynth-train-2.tfrecord", 
                 "E:/NSynth/nsynth-train-3.tfrecord",
                 "E:/NSynth/nsynth-train-4.tfrecord",
                 "E:/NSynth/nsynth-train-5.tfrecord",
                 "E:/NSynth/nsynth-train-6.tfrecord",
                 "E:/NSynth/nsynth-train-7.tfrecord",
                 "E:/NSynth/nsynth-train-8.tfrecord",
                 "E:/NSynth/nsynth-train-9.tfrecord",
                 "E:/NSynth/nsynth-train-10.tfrecord"]
TEST_DATA = "E:/NSynth/nsynth-valid.tfrecord"
EVAL_DATA = "E:/NSynth/nsynth-test.tfrecord"
MODEL_PATH = "E:/NSynth/trained_models_2/"

# Data parameters
SAMPLE_RATE = 16000

In [3]:
# Test GPU presence
# If no errors thrown, GPU is being used
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

[[22. 28.]
 [49. 64.]]


In [4]:
# Create the neural network
def conv_net(x_dict, n_classes, dropout, reuse, is_training):
    
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):
        # TF Estimator input is a dict, in case of multiple inputs
        x = x_dict['image']

        # Spectrogram data input is a 1-D vector of 16384 features (128*128 pixels)
        # Reshape to match picture format [Height x Width x Channel]
        # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
        x = tf.reshape(x, shape=[-1, 128, 128, 1])

        # Convolution Layer with 32 filters and a kernel size of 5
        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

        # Convolution Layer with 64 filters and a kernel size of 3
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)

        # Flatten the data to a 1-D vector for the fully connected layer
        fc1 = tf.contrib.layers.flatten(conv2)

        # Fully connected layer (in tf contrib folder for now)
        fc1 = tf.layers.dense(fc1, 1024)
        # Apply Dropout (if is_training is False, dropout is not applied)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)

        # Output layer, class prediction
        out = tf.layers.dense(fc1, n_classes)

    return out

In [5]:
# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    
    # Build the neural network
    # Because Dropout have different behavior at training and prediction time, we
    # need to create 2 distinct computation graphs that still share the same weights.
    logits_train = conv_net(features, NUM_CLASSES, DROPOUT, reuse=False, is_training=True)
    logits_test = conv_net(features, NUM_CLASSES, DROPOUT, reuse=True, is_training=False)
    
    # Predictions
    pred_classes = tf.argmax(logits_test, axis=1)
    pred_probas = tf.nn.softmax(logits_test)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 
        
    # Define loss and optimizer
    labels = tf.reshape(labels, shape=[-1]) # Squeeze labels from 2D to 1D - don't know why they are fed as 2D
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [6]:
# Build the Estimator
model = tf.estimator.Estimator(model_fn, model_dir=MODEL_PATH)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'E:/NSynth/trained_models_2/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000027D132E4588>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
# Function to parse the TFRecord
def _parse_(serialized_example):
    feature_list = {
                'audio': tf.FixedLenSequenceFeature(shape=[1], dtype=tf.float32, allow_missing=True),
                'instrument_family': tf.FixedLenSequenceFeature(shape=[], dtype=tf.int64, allow_missing=True),
}
    
    # Extract example by features
    example = tf.parse_single_example(serialized_example, feature_list)
    
    # Convert audio data to normalized spectrogram - time (4sec) vs frequency intensity
    # Spectrogram contains values [-1, 1] for intensity, is 128 x 128 in resolution
    spectrogram = audio_ops.audio_spectrogram(example["audio"], window_size=1024, stride=64)
    max_val = tf.reduce_max(spectrogram, axis=None)
    min_const = tf.constant(2.)
    brightness_const = tf.divide(min_const, max_val)   # Calculate normalization constant
    brightened_spect = tf.multiply(spectrogram, brightness_const)
    minned_spectrogram = tf.minimum(brightened_spect, min_const) # Remove other spikes?
    shifted_spectrogram = tf.subtract(minned_spectrogram, tf.constant(1.0, dtype=tf.float32))
    expanded = tf.expand_dims(shifted_spectrogram, -1)
    resized = tf.image.resize_bilinear(expanded, [128, 128])
    squeezed = tf.squeeze(resized, 0)
    flipped = tf.image.flip_left_right(squeezed)
    normalized_spectrogram = tf.image.transpose_image(flipped)
    
    # Cast data to input format required by model
    image = normalized_spectrogram
    label = example['instrument_family']
    return (dict({'image':image}),label)



# Define the data input function for training
def tfrecord_train_input_fn(batch_size=BATCH_SIZE):
    files = tf.data.Dataset.from_tensor_slices(TRAINING_DATA)
    tfrecord_dataset = files.interleave(lambda x: tf.data.TFRecordDataset(x), cycle_length=11, block_length=1)
    tfrecord_dataset = tfrecord_dataset.map(lambda   x:_parse_(x)).shuffle(True).batch(batch_size)
    tfrecord_iterator = tfrecord_dataset.make_one_shot_iterator()
    
    return tfrecord_iterator.get_next()

# Train the Model
# With debug
# hooks = [tf_debug.LocalCLIDebugHook(ui_type="readline")]
# model.train(tfrecord_train_input_fn, steps=NUM_STEPS, hooks=hooks)
# without debug
model.train(tfrecord_train_input_fn, steps=NUM_STEPS)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into E:/NSynth/trained_models_2/model.ckpt.
INFO:tensorflow:loss = 2.4262404, step = 0
INFO:tensorflow:global_step/sec: 2.09135
INFO:tensorflow:loss = 1.7915858, step = 100 (47.817 sec)
INFO:tensorflow:global_step/sec: 2.10677
INFO:tensorflow:loss = 0.96903753, step = 200 (47.466 sec)
INFO:tensorflow:global_step/sec: 2.10249
INFO:tensorflow:loss = 1.5704811, step = 300 (47.611 sec)
INFO:tensorflow:global_step/sec: 2.10539
INFO:tensorflow:loss = 1.2587106, step = 400 (47.449 sec)
INFO:tensorflow:global_step/sec: 2.10886
INFO:tensorflow:loss = 1.0862541, step = 500 (47.419 sec)
INFO:tensorflow:global_step/sec: 2.09459
INFO:tensorflow:loss = 1.4213684, step = 600 (47.792 sec)
INFO:tensorflow:global_step/sec: 2.10753

INFO:tensorflow:loss = 1.0944868, step = 7500 (45.614 sec)
INFO:tensorflow:Saving checkpoints for 7563 into E:/NSynth/trained_models_2/model.ckpt.
INFO:tensorflow:global_step/sec: 1.74793
INFO:tensorflow:loss = 5.004697, step = 7600 (57.167 sec)
INFO:tensorflow:global_step/sec: 2.27333
INFO:tensorflow:loss = 0.71625483, step = 7700 (43.988 sec)
INFO:tensorflow:global_step/sec: 2.25671
INFO:tensorflow:loss = 0.6991025, step = 7800 (44.348 sec)
INFO:tensorflow:global_step/sec: 2.25656
INFO:tensorflow:loss = 2.3382692, step = 7900 (44.279 sec)
INFO:tensorflow:global_step/sec: 2.2478
INFO:tensorflow:loss = 1.0858443, step = 8000 (44.488 sec)
INFO:tensorflow:global_step/sec: 2.25576
INFO:tensorflow:loss = 0.71379507, step = 8100 (44.376 sec)
INFO:tensorflow:global_step/sec: 2.2463
INFO:tensorflow:loss = 0.8351855, step = 8200 (44.473 sec)
INFO:tensorflow:global_step/sec: 2.26415
INFO:tensorflow:loss = 0.7084348, step = 8300 (44.167 sec)
INFO:tensorflow:global_step/sec: 2.24957
INFO:tensorfl

In [8]:
# Evaluate the Model
# Define the input function for validation
def tfrecord_valid_input_fn(batch_size=BATCH_SIZE):
    tfrecord_dataset = tf.data.TFRecordDataset(TEST_DATA)
    tfrecord_dataset = tfrecord_dataset.map(lambda   x:_parse_(x)).shuffle(True).batch(batch_size)
    tfrecord_iterator = tfrecord_dataset.make_one_shot_iterator()
    return tfrecord_iterator.get_next()

# Define the input function for evaluation
def tfrecord_eval_input_fn(batch_size=BATCH_SIZE):
    tfrecord_dataset = tf.data.TFRecordDataset(EVAL_DATA)
    tfrecord_dataset = tfrecord_dataset.map(lambda   x:_parse_(x)).shuffle(True).batch(batch_size)
    tfrecord_iterator = tfrecord_dataset.make_one_shot_iterator()
    return tfrecord_iterator.get_next()


# Validate the model
model.evaluate(tfrecord_valid_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-11-01:14:32
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models_2/model.ckpt-9038
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-11-01:17:09
INFO:tensorflow:Saving dict for global step 9038: accuracy = 0.16437924, global_step = 9038, loss = 6.0761247
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 9038: E:/NSynth/trained_models_2/model.ckpt-9038


{'accuracy': 0.16437924, 'loss': 6.0761247, 'global_step': 9038}

In [9]:
# Epoch 2
model.train(tfrecord_train_input_fn, steps=NUM_STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models_2/model.ckpt-9038
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 9038 into E:/NSynth/trained_models_2/model.ckpt.
INFO:tensorflow:loss = 4.9295707, step = 9038
INFO:tensorflow:global_step/sec: 2.0909
INFO:tensorflow:loss = 1.7092929, step = 9138 (47.826 sec)
INFO:tensorflow:global_step/sec: 2.08153
INFO:tensorflow:loss = 1.4212272, step = 9238 (48.042 sec)
INFO:tensorflow:global_step/sec: 1.95843
INFO:tensorflow:loss = 1.4184024, step = 9338 (51.133 sec)
INFO:tensorflow:global_step/sec: 2.09499
INFO:tensorflow:loss = 1.1631222, step = 9438 (47.662 sec)
INFO:tensorflow:global_step/sec: 2.07731
INFO:tensorflow:loss = 0.86078346, step = 9538 (48.140 sec)
INFO:tensorflow:global_step/sec: 2.09506
INF

INFO:tensorflow:loss = 1.3330265, step = 16338 (45.289 sec)
INFO:tensorflow:global_step/sec: 2.20711
INFO:tensorflow:loss = 0.7094647, step = 16438 (45.308 sec)
INFO:tensorflow:global_step/sec: 2.18881
INFO:tensorflow:loss = 1.1066661, step = 16538 (45.737 sec)
INFO:tensorflow:Saving checkpoints for 16542 into E:/NSynth/trained_models_2/model.ckpt.
INFO:tensorflow:global_step/sec: 1.73626
INFO:tensorflow:loss = 1.0364534, step = 16638 (57.545 sec)
INFO:tensorflow:global_step/sec: 2.26706
INFO:tensorflow:loss = 0.012632469, step = 16738 (44.110 sec)
INFO:tensorflow:global_step/sec: 2.24988
INFO:tensorflow:loss = 0.7738309, step = 16838 (44.488 sec)
INFO:tensorflow:global_step/sec: 2.25156
INFO:tensorflow:loss = 1.7924986, step = 16938 (44.373 sec)
INFO:tensorflow:global_step/sec: 2.25999
INFO:tensorflow:loss = 0.85830086, step = 17038 (44.248 sec)
INFO:tensorflow:global_step/sec: 2.25043
INFO:tensorflow:loss = 0.6558316, step = 17138 (44.474 sec)
INFO:tensorflow:global_step/sec: 2.2541


In [10]:
# Validate the model
model.evaluate(tfrecord_valid_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-11-13:57:48
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models_2/model.ckpt-18076
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-11-14:00:25
INFO:tensorflow:Saving dict for global step 18076: accuracy = 0.16398485, global_step = 18076, loss = 6.223574
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 18076: E:/NSynth/trained_models_2/model.ckpt-18076


{'accuracy': 0.16398485, 'loss': 6.223574, 'global_step': 18076}

In [11]:
# Epoch 3
model.train(tfrecord_train_input_fn, steps=NUM_STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models_2/model.ckpt-18076
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 18076 into E:/NSynth/trained_models_2/model.ckpt.
INFO:tensorflow:loss = 5.2037745, step = 18076
INFO:tensorflow:global_step/sec: 2.09217
INFO:tensorflow:loss = 1.731421, step = 18176 (47.799 sec)
INFO:tensorflow:global_step/sec: 2.09664
INFO:tensorflow:loss = 1.5096297, step = 18276 (47.695 sec)
INFO:tensorflow:global_step/sec: 2.09131
INFO:tensorflow:loss = 1.6396569, step = 18376 (47.868 sec)
INFO:tensorflow:global_step/sec: 2.08516
INFO:tensorflow:loss = 1.5536594, step = 18476 (47.906 sec)
INFO:tensorflow:global_step/sec: 2.08584
INFO:tensorflow:loss = 1.3165166, step = 18576 (47.941 sec)
INFO:tensorflow:global_step/sec: 2.09

INFO:tensorflow:loss = 1.2922623, step = 25376 (45.382 sec)
INFO:tensorflow:global_step/sec: 2.18928
INFO:tensorflow:loss = 0.6735517, step = 25476 (45.676 sec)
INFO:tensorflow:global_step/sec: 2.18284
INFO:tensorflow:loss = 1.1323526, step = 25576 (45.843 sec)
INFO:tensorflow:Saving checkpoints for 25614 into E:/NSynth/trained_models_2/model.ckpt.
INFO:tensorflow:global_step/sec: 1.74638
INFO:tensorflow:loss = 1.2207328, step = 25676 (57.230 sec)
INFO:tensorflow:global_step/sec: 2.26856
INFO:tensorflow:loss = 0.28937745, step = 25776 (44.081 sec)
INFO:tensorflow:global_step/sec: 2.26393
INFO:tensorflow:loss = 0.7053685, step = 25876 (44.207 sec)
INFO:tensorflow:global_step/sec: 2.25089
INFO:tensorflow:loss = 2.0395033, step = 25976 (44.391 sec)
INFO:tensorflow:global_step/sec: 2.25793
INFO:tensorflow:loss = 1.0128155, step = 26076 (44.289 sec)
INFO:tensorflow:global_step/sec: 2.25852
INFO:tensorflow:loss = 0.686508, step = 26176 (44.305 sec)
INFO:tensorflow:global_step/sec: 2.25562
IN

In [12]:
# Validate the model
model.evaluate(tfrecord_valid_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-11-15:12:20
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models_2/model.ckpt-27114
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-11-15:14:57
INFO:tensorflow:Saving dict for global step 27114: accuracy = 0.16430037, global_step = 27114, loss = 6.5358233
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 27114: E:/NSynth/trained_models_2/model.ckpt-27114


{'accuracy': 0.16430037, 'loss': 6.5358233, 'global_step': 27114}

In [13]:
# Epoch 4
model.train(tfrecord_train_input_fn, steps=NUM_STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models_2/model.ckpt-27114
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 27114 into E:/NSynth/trained_models_2/model.ckpt.
INFO:tensorflow:loss = 5.7159343, step = 27114
INFO:tensorflow:global_step/sec: 2.10244
INFO:tensorflow:loss = 1.722009, step = 27214 (47.566 sec)
INFO:tensorflow:global_step/sec: 2.09059
INFO:tensorflow:loss = 1.5208197, step = 27314 (47.832 sec)
INFO:tensorflow:global_step/sec: 2.08759
INFO:tensorflow:loss = 1.512239, step = 27414 (47.948 sec)
INFO:tensorflow:global_step/sec: 2.08963
INFO:tensorflow:loss = 1.7337084, step = 27514 (47.809 sec)
INFO:tensorflow:global_step/sec: 2.10541
INFO:tensorflow:loss = 1.8271294, step = 27614 (47.497 sec)
INFO:tensorflow:global_step/sec: 2.083

INFO:tensorflow:loss = 1.333297, step = 34414 (45.565 sec)
INFO:tensorflow:global_step/sec: 2.18744
INFO:tensorflow:loss = 0.715647, step = 34514 (45.716 sec)
INFO:tensorflow:global_step/sec: 2.19017
INFO:tensorflow:loss = 1.1050938, step = 34614 (45.717 sec)
INFO:tensorflow:Saving checkpoints for 34661 into E:/NSynth/trained_models_2/model.ckpt.
INFO:tensorflow:global_step/sec: 1.74432
INFO:tensorflow:loss = 1.0619951, step = 34714 (57.269 sec)
INFO:tensorflow:global_step/sec: 2.26722
INFO:tensorflow:loss = 0.72481894, step = 34814 (44.108 sec)
INFO:tensorflow:global_step/sec: 2.25343
INFO:tensorflow:loss = 0.68408465, step = 34914 (44.412 sec)
INFO:tensorflow:global_step/sec: 2.24708
INFO:tensorflow:loss = 2.4459953, step = 35014 (44.466 sec)
INFO:tensorflow:global_step/sec: 2.2604
INFO:tensorflow:loss = 1.0376041, step = 35114 (44.240 sec)
INFO:tensorflow:global_step/sec: 2.25558
INFO:tensorflow:loss = 0.71888554, step = 35214 (44.374 sec)
INFO:tensorflow:global_step/sec: 2.24675
IN

In [14]:
# Validate the model
model.evaluate(tfrecord_valid_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-11-16:26:47
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models_2/model.ckpt-36152
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-11-16:29:22
INFO:tensorflow:Saving dict for global step 36152: accuracy = 0.16430037, global_step = 36152, loss = 6.0176644
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 36152: E:/NSynth/trained_models_2/model.ckpt-36152


{'accuracy': 0.16430037, 'loss': 6.0176644, 'global_step': 36152}

In [15]:
# Epoch 5
model.train(tfrecord_train_input_fn, steps=NUM_STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models_2/model.ckpt-36152
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 36152 into E:/NSynth/trained_models_2/model.ckpt.
INFO:tensorflow:loss = 5.10375, step = 36152
INFO:tensorflow:global_step/sec: 2.10941
INFO:tensorflow:loss = 1.720696, step = 36252 (47.408 sec)
INFO:tensorflow:global_step/sec: 2.09953
INFO:tensorflow:loss = 1.640273, step = 36352 (47.630 sec)
INFO:tensorflow:global_step/sec: 2.09455
INFO:tensorflow:loss = 1.646882, step = 36452 (47.816 sec)
INFO:tensorflow:global_step/sec: 2.0908
INFO:tensorflow:loss = 1.7761912, step = 36552 (47.756 sec)
INFO:tensorflow:global_step/sec: 2.09816
INFO:tensorflow:loss = 1.8097414, step = 36652 (47.662 sec)
INFO:tensorflow:global_step/sec: 2.10458
I

INFO:tensorflow:loss = 1.3542128, step = 43452 (45.448 sec)
INFO:tensorflow:global_step/sec: 2.18981
INFO:tensorflow:loss = 0.70068955, step = 43552 (45.665 sec)
INFO:tensorflow:global_step/sec: 2.18693
INFO:tensorflow:loss = 1.1133518, step = 43652 (45.777 sec)
INFO:tensorflow:Saving checkpoints for 43701 into E:/NSynth/trained_models_2/model.ckpt.
INFO:tensorflow:global_step/sec: 1.74494
INFO:tensorflow:loss = 1.0154619, step = 43752 (57.257 sec)
INFO:tensorflow:global_step/sec: 2.26222
INFO:tensorflow:loss = 0.72521985, step = 43852 (44.204 sec)
INFO:tensorflow:global_step/sec: 2.2474
INFO:tensorflow:loss = 0.7000307, step = 43952 (44.540 sec)
INFO:tensorflow:global_step/sec: 2.25391
INFO:tensorflow:loss = 2.2364745, step = 44052 (44.323 sec)
INFO:tensorflow:global_step/sec: 2.25866
INFO:tensorflow:loss = 1.0992956, step = 44152 (44.274 sec)
INFO:tensorflow:global_step/sec: 2.24464
INFO:tensorflow:loss = 0.71776664, step = 44252 (44.596 sec)
INFO:tensorflow:global_step/sec: 2.24695


In [16]:
# Validate the model
model.evaluate(tfrecord_valid_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-11-17:41:12
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models_2/model.ckpt-45190
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-11-17:43:48
INFO:tensorflow:Saving dict for global step 45190: accuracy = 0.16437924, global_step = 45190, loss = 8.512051
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 45190: E:/NSynth/trained_models_2/model.ckpt-45190


{'accuracy': 0.16437924, 'loss': 8.512051, 'global_step': 45190}

In [ ]:
# Epoch 6
model.train(tfrecord_train_input_fn, steps=NUM_STEPS)

In [ ]:
# Validate the model
model.evaluate(tfrecord_valid_input_fn)

In [ ]:
# Evaluate the model
model.evaluate(tfrecord_eval_input_fn)